#Create aggregated tables
 
Objective: Analyze Northwind data using fact and dimension tables.

## Configuration

Before executing this cell, add your name to the file:
<a href="$./includes/configuration" target="_blank">
includes/configuration</a>

```username = "your_name"```
```                      ```

In [0]:
%run ./includes/configuration

Out[3]: DataFrame[]

## Create aggregated tables

**Note:**
In this sample project, files are saved to Databricks File System (DBFS).
The good practice is to save files in a cloud based storage. DBFS is used only for demo purpose.

#### Step 0: Recoverr refined tables
Refined tables were saved in `refined_path` folder

In [0]:
for table_name in northwind_tables_refined:
  spark.sql(
    f"""
    DROP TABLE IF EXISTS {table_name}
    """
  )

In [0]:
for table_name in northwind_tables_refined:
  spark.sql(
    f"""
    CREATE TABLE {table_name}
    USING DELTA
    LOCATION "{refined_path}{table_name}"
    """
  )

#### Step 1: Describe refined tables
Before aggregating refined tables, let us check if attribute `Provider` is `DELTA`. The Spark SQL command `DESCRIBE`, with optioinal parameter `EXTENDED`, is used to collect the attribute.

In [0]:
for table_name in northwind_tables_refined:
  df = spark.sql(
    f"""
    DESCRIBE EXTENDED {table_name};
    """
    )
  provider = df.filter(df.col_name == 'Provider').collect()[0][1].upper()
  assert provider == 'DELTA', f"provider is {provider}, not DELTA"
print("Assertion passed - provider is DELTA.")

Assertion passed - provider is DELTA.


In [0]:
display(
  spark.sql(
    f"""
    DESCRIBE EXTENDED ft_orders;
    """
  )
)

col_name,data_type,comment
order_id,smallint,
customer_id,string,
employee_id,smallint,
order_date,date,YYYY-mm-dd
order_year,string,YYYY
order_month,string,mm
order_day,string,dd
required_date,date,YYYY-mm-dd
shipped_date,date,YYYY-mm-dd
ship_via,smallint,


### Create new delta talbes with aggregation
Aggregated tables are created from Delta tables recovered in step 0.
In the context of EDSS (Enterprise Decision Support System), these are downstream agrregated tables or data mart.

#### Step 1: Remove files in `user_analytics` folder
This step assures the notebook is idempotent. It means the notebook can be executed multiple times and the result will be same - no error raised nor extra files saved.

In [0]:
dbutils.fs.rm(analytics_path,
              recurse=True)

Out[14]: False

#### Substeps 1 to 3 are repeated to answer few business questions
- Substep 1: create aggreagated DataFrame
- Substep 2: save Delta file
- Substep 3: register Delta table in Metastore and execute the query

In [0]:
from pyspark.sql.functions import *

##### Question 1
- What are the three products with the lowest sales?

###### 1.1 - Create aggregated DataFrame

In [0]:
df_orders = spark.read.table('ft_orders')

user_analytics_1 = (
  df_orders
  .groupby("product_id")
  .agg(sum(col("quantity")).alias("sum_quantity"),
       avg(col("quantity")).alias("avg_quantity"),
       min(col("quantity")).alias("min_quantity"),
       max(col("quantity")).alias("max_quantity"))
)

###### 1.2 - Save Delta file

In [0]:
(user_analytics_1.write
 .format("delta")
 .mode("overwrite")
 .save(analytics_path + 'user_analytics_1'))

###### 1.3 - Register Delta table in Metastore and execute the query

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS user_analytics_1
""")

spark.sql(f"""
CREATE TABLE user_analytics_1
USING DELTA
LOCATION "{analytics_path + 'user_analytics_1'}"
""")


Out[18]: DataFrame[]

In [0]:
%sql
SELECT product_id, sum_quantity
FROM user_analytics_1
ORDER BY sum_quantity ASC
LIMIT 3

product_id,sum_quantity
9,95
15,122
37,125


##### Questions 2 e 3
- Who are the five clients with more orders?
- Who are the five clients with the highest total amounts?

###### 2.1 - Create aggregated DataFrame

In [0]:
user_analytics_2 = (
  df_orders
  .groupby("customer_id", "order_id")
  .agg(sum((col('unit_price') * col('quantity') - col('discount'))).alias('order_total'))
)

###### 2.2 - Save Delta file

In [0]:
(user_analytics_2.write
 .format("delta")
 .mode("overwrite")
 .save(analytics_path + 'user_analytics_2'))

###### 2.3 - Register Delta table in Metastore and execute the query

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS user_analytics_2
""")

spark.sql(f"""
CREATE TABLE user_analytics_2
USING DELTA
LOCATION "{analytics_path + 'user_analytics_2'}"
""")


Out[21]: DataFrame[]

Question 2 - Who are the five clients with more orders?

In [0]:
%sql
SELECT
  customer_id,
  COUNT(order_id) AS number_orders
FROM user_analytics_2
GROUP BY customer_id
ORDER BY number_orders DESC
LIMIT 5

customer_id,number_orders
SAVEA,31
ERNSH,30
QUICK,28
FOLKO,19
HUNGO,19


Question 3 - Who are the five clients with the highest total amounts?
<br><br>**Considered the total amount per order.**

In [0]:
%sql
SELECT *
FROM user_analytics_2
ORDER BY order_total DESC
LIMIT 5

customer_id,order_id,order_total
QUICK,10865,17249.900146484375
SAVEA,11030,16321.14990234375
HANAR,10981,15810.0
QUEEN,10372,12280.200012207031
MEREP,10424,11492.599975585938


##### Question 4
- Who is the best employee in the last registered month? (total sales)

###### 3.1 - Create aggregated DataFrame

In [0]:
df_orders = spark.read.table('ft_orders')

user_analytics_3 = (
  df_orders
  .groupby("employee_id", "order_year", "order_month")
  .agg(sum((col('unit_price') * col('quantity') - col('discount'))).alias('orders_total'))
)

###### 3.2 - Save Delta file

In [0]:
(user_analytics_3.write
 .format("delta")
 .mode("overwrite")
 .save(analytics_path + 'user_analytics_3'))

###### 3.3 - Register Delta table in Metastore and execute the query

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS user_analytics_3
""")

spark.sql(f"""
CREATE TABLE user_analytics_3
USING DELTA
LOCATION "{analytics_path + 'user_analytics_3'}"
""")


Out[24]: DataFrame[]

In [0]:
%sql
SELECT *
FROM user_analytics_3
ORDER BY order_year DESC, order_month DESC, orders_total DESC
LIMIT 1

employee_id,order_year,order_month,orders_total
1,1998,5,7052.659925460815


##### Question 5
- What are regions with less registered clients?

###### 4.1 - Create aggregated DataFrame

In [0]:
df_customers = spark.read.table('dm_customers')

user_analytics_4 = (
  df_customers
  .groupby("region")
  .agg(count("customer_id").alias("customers_count"))
)

###### 4.2 - Save Delta file

In [0]:
(user_analytics_4.write
 .format("delta")
 .mode("overwrite")
 .save(analytics_path + 'user_analytics_4'))

###### 4.3 - Register Delta table in Metastore and execute the query

In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS user_analytics_4
""")

spark.sql(f"""
CREATE TABLE user_analytics_4
USING DELTA
LOCATION "{analytics_path + 'user_analytics_4'}"
""")


Out[27]: DataFrame[]

In [0]:
%sql
SELECT region, customers_count
FROM user_analytics_4
ORDER BY customers_count ASC

region,customers_count
MT,1
NM,1
Québec,1
WY,1
Táchira,1
DF,1
Lara,1
AK,1
Co. Cork,1
ID,1
